### Web Scraping de ofertas de Programación en Bumeran

Este notebook implementa un scraping de ofertas de trabajo en Bumeran, aplicando filtros específicos:  
- Fecha de publicación: menor a 15 días  
- Área: Tecnología, sistemas y telecomunicación  
- Subárea: Programación  
- Carga horaria: Full-Time  
- Departamento: Lima  


In [1]:
#Paso 1: instalamos librerías requeridas
!pip install selenium beautifulsoup4 pandas

In [2]:
!pip install webdriver-manager

  Obtaining dependency information for webdriver-manager from https://files.pythonhosted.org/packages/b5/b5/3bd0b038d80950ec13e6a2c8d03ed8354867dc60064b172f2f4ffac8afbe/webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata


In [37]:
#Paso 2: importamos las librerías

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [47]:
# Configurar el driver (ANTIGUO)
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # para no abrir ventana (opcional)
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [52]:
# Configurar el driver (NUEVO)
def setup_stealth_driver():
    """Configurar el driver con técnicas anti-detección"""
    options = webdriver.ChromeOptions()
    # ⚠️ IMPORTANTE: Comentar --headless para evitar detección
    # options.add_argument("--headless")
    
    # User agent real
    options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    
    # Configuraciones anti-detección
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-plugins")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # Ejecutar scripts anti-detección
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    driver.execute_script("Object.defineProperty(navigator, 'plugins', {get: () => [1, 2, 3, 4, 5]})")
    driver.execute_script("Object.defineProperty(navigator, 'languages', {get: () => ['es-ES', 'es']})")
    driver.execute_script("window.chrome = {runtime: {}}")
    
    return driver

# Configurar driver con técnicas stealth
print("🥷 Configurando driver con técnicas anti-detección...")
driver = setup_stealth_driver()
print("✅ Driver stealth configurado correctamente")

🥷 Configurando driver con técnicas anti-detección...
✅ Driver stealth configurado correctamente


In [48]:
#Paso 3: Definir filtros y construir URL
# Filtros configurables
departamento = "lima"
area = "tecnologia-sistemas-y-telecomunicaciones"
subarea = "programacion"
carga = "full-time"
fecha = "menor-a-15-dias"

# Construcción dinámica del URL
base_url = (
    f"https://www.bumeran.com.pe/en-{departamento}/"
    f"empleos-area-{area}-subarea-{subarea}-{carga}-publicacion-{fecha}.html"
)

print("URL generada con filtros:", base_url)

URL generada con filtros: https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-full-time-publicacion-menor-a-15-dias.html


In [53]:
#Detectar número de páginas con Selenium

driver = webdriver.Chrome()
driver.get(base_url)
time.sleep(3)  # esperar a que cargue JS

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Ahora sí debería encontrar todas las páginas
pages = [a.text for a in soup.select("a[href*='?page=']")]
print(pages)

['2', '3', '4', '5', '6', '']


In [54]:
# Obtener la última página
max_page = max(pages) if pages else 1
print(f"Se detectaron {max_page} páginas de resultados.")

Se detectaron 6 páginas de resultados.


In [55]:
#Recolectar links de ofertas

job_links = set()

for page in range(1, int(max_page) + 1):
    #Construir la URL según la página
    url = base_url if page == 1 else f"{base_url}?page={page}"
    print(f"Scraping página {page}/{max_page}...")

    driver.get(url)
    time.sleep(8)  # espera a que cargue la página

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Buscar todos los links de ofertas
    for a in soup.select("a[href*='/empleos/']"):
        link = a["href"]
        if link.startswith("/"):
            link = "https://www.bumeran.com.pe" + link
        job_links.add(link)

print(f"Total de ofertas recolectadas: {len(job_links)}")

Scraping página 1/6...
Scraping página 2/6...
Scraping página 3/6...
Scraping página 4/6...
Scraping página 5/6...
Scraping página 6/6...
Total de ofertas recolectadas: 105


In [56]:
import random

def extract_job_details_stealth(driver, job_url):
    """Extraer detalles con técnicas anti-detección"""
    try:
        # Pausa aleatoria entre 3-8 segundos para simular comportamiento humano
        wait_time = random.uniform(3, 8)
        print(f"   ⏳ Esperando {wait_time:.1f}s antes de acceder...")
        time.sleep(wait_time)
        
        # Navegar a la página
        driver.get(job_url)
        
        # Esperar más tiempo para que cargue completamente
        time.sleep(random.uniform(5, 10))
        
        # Verificar si fuimos bloqueados
        page_source = driver.page_source
        if "blocked" in page_source.lower() or "captcha" in page_source.lower():
            print("   🚨 ¡Detectado bloqueo! Esperando más tiempo...")
            time.sleep(random.uniform(15, 25))
            driver.refresh()
            time.sleep(10)
            page_source = driver.page_source
        
        soup = BeautifulSoup(page_source, "html.parser")
        
        # 1. EXTRAER TÍTULO usando los selectores que identificaste
        titulo = None
        title_selectors = [
            "h1.sc-iHrGRV.gplQEj",  # Selector específico que encontraste
            "h1.sc-iHrGRV",         # Versión más general
            "h1[class*='sc-iHrGRV']", # Con wildcards
            "h1",                    # Fallback
            "[data-testid='job-title']"
        ]
        
        for selector in title_selectors:
            try:
                title_element = soup.select_one(selector)
                if title_element and title_element.get_text(strip=True):
                    titulo = title_element.get_text(strip=True)
                    print(f"   📝 Título encontrado: {titulo[:50]}...")
                    break
            except:
                continue

        return {
            "Titulo": titulo
            }
    
    except Exception as e:
        print(f"   ❌ Error procesando: {str(e)}")
        return {
            "Titulo": f"Error: {str(e)}"
        }

In [57]:
#CELDA 8: Procesar todas las ofertas (esta celda tomará tiempo)
print("🚀 Iniciando extracción de detalles...")
jobs = []

for idx, job_url in enumerate(job_links, start=1):
    print(f"[{idx}/{len(job_links)}] Procesando: {job_url.split('/')[-1]}")
    
    job_details = extract_job_details(driver, job_url)
    jobs.append(job_details)
    
    # Mostrar progreso cada 10 ofertas
    if idx % 10 == 0:
        successful = len([j for j in jobs if j['Titulo'] and not j['Titulo'].startswith('Error')])
        print(f"  ✅ Progreso: {idx}/{len(job_links)} | Exitosas: {successful}")
    
    # Pausa entre requests
    time.sleep(1)

print(f"✅ Procesamiento completado. Total: {len(jobs)} ofertas")

🚀 Iniciando extracción de detalles...
[1/105] Procesando: backend-java-sector-bancario-experis-peru-1117965923.html
[2/105] Procesando: analista-senior-de-automatizaciones-e-inteligencia-artificial-hibrido-cibergestion-peru-1117964801.html
[3/105] Procesando: programador-1117960615.html
[4/105] Procesando: desarrollador-fullstack-java-js-remoto-protiviti-peru-1117966558.html
[5/105] Procesando: analista-de-programacion-rpg-izipay-1117080379.html
[6/105] Procesando: programador-ologgi-s.a.c-1117973857.html
[7/105] Procesando: analista-programador-postgre-oracle-hibrido-green-solutions-1117962598.html
[8/105] Procesando: desarrollador-backend-inetum-peru-1117968206.html
[9/105] Procesando: programador-web-y-movil-manpowergroup-peru-1117958854.html
[10/105] Procesando: analista-programador-java-1117972678.html
  ✅ Progreso: 10/105 | Exitosas: 10
[11/105] Procesando: desarrollador-python-odoo-ibr-peru-s.a.-1117961539.html
[12/105] Procesando: desarrollador-backend-javascript-senior-remoto-

[95/105] Procesando: analista-programador-java-oracle-presencial-hitss-peru-1117963095.html
[96/105] Procesando: desarrollador-fullstack-semiseniorjava-angular-csti-corp-1117961713.html
[97/105] Procesando: desarrollador-backend-nodejs-senior-zoluxiones-sac-1117977115.html
[98/105] Procesando: desarrollador-.net-php-indra-peru-1117961552.html
[99/105] Procesando: practicante-profesional-programacion-plsql-indra-peru-1117960022.html
[100/105] Procesando: desarrolladora-mineria-lurin-zamine-peru-1117976772.html
  ✅ Progreso: 100/105 | Exitosas: 100
[101/105] Procesando: analista-programador-java-remoto-planilla-completa-green-solutions-1117971817.html
[102/105] Procesando: desarrollador-fullstack-java-hitss-peru-1117967606.html
[103/105] Procesando: programador-frontend-senior-sonda-del-peru-s.a.-1117973000.html
[104/105] Procesando: analista-de-imagenes-hospitalarias-sistemas-grupo-san-pablo-1117977289.html
[105/105] Procesando: desarrollador-backend-.net-indra-peru-1117971369.html
✅ Pr

In [58]:
# Crear DataFrame
df2 = pd.DataFrame(jobs)
df2.to_csv("bumeran_jobs_ejemplo.csv", index=False, encoding="utf-8-sig")